In [1]:
!ls

data  keras spiking.ipynb  keras_to_snn_params.npz  tf to snn converter.ipynb


In [2]:
# !rm -rf data/

In [3]:
# !unzip ../data.zip

Archive:  ../data.zip
   creating: data/
   creating: data/dep/
  inflating: data/dep/epoch-000.png  
  inflating: data/dep/epoch-000_run.png  
  inflating: data/dep/epoch-001.png  
  inflating: data/dep/epoch-001_run.png  
  inflating: data/dep/epoch-002.png  
  inflating: data/dep/epoch-002_run.png  
  inflating: data/dep/epoch-003.png  
  inflating: data/dep/epoch-003_run.png  
  inflating: data/dep/epoch-004.png  
  inflating: data/dep/epoch-004_run.png  
  inflating: data/dep/epoch-005.png  
  inflating: data/dep/epoch-005_run.png  
  inflating: data/dep/epoch-006.png  
  inflating: data/dep/epoch-006_run.png  
  inflating: data/dep/epoch-007.png  
  inflating: data/dep/epoch-007_run.png  
  inflating: data/dep/epoch-008.png  
  inflating: data/dep/epoch-008_run.png  
  inflating: data/dep/epoch-009.png  
  inflating: data/dep/epoch-009_run.png  
  inflating: data/dep/epoch-010.png  
  inflating: data/dep/epoch-010_run.png  
  inflating: data/dep/epoch-011.png  
  inflating: data/

In [4]:
!pip install tensorflow --user

  Using cached https://files.pythonhosted.org/packages/08/d6/a6aaa29fea945bc6c61d11f6e0697b325ff7446de5ffd62c2fa02f627048/numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl


In [5]:
!pip install keras-spiking --user

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import keras_spiking

tf.random.set_seed(0)
np.random.seed(0)

In [ ]:
%config Completer.use_jedi = False

In [ ]:
img_height = 256
img_width = 256

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  './data/',
  validation_split=0.2,
  subset="training",
#   color_mode = 'grayscale',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=32)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  './data/',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=32)


In [ ]:
class_names = train_ds.class_names

In [ ]:
x_train, y_train = [], []
x_test, y_test = [], []
for images, labels in train_ds:
  x_train.append(images.numpy())
#   y_train.append(labels.numpy())
x_train = np.concatenate(x_train, axis = 0) 
# np.array(x_train).shape >> (232,432,288,3)

for images, labels in val_ds:
  x_test.append(images.numpy())
#   y_test.append(labels.numpy())
x_test = np.concatenate(x_test, axis = 0) 

y_train = np.concatenate([y for x, y in train_ds], axis=0)
y_test = np.concatenate([y for x, y in val_ds], axis=0)

In [ ]:
x_train = tf.image.central_crop(x_train, 0.6)
x_test = tf.image.central_crop(x_test, 0.6)

In [ ]:
x_train /= 255.
x_test /= 255.

In [ ]:
x_test.shape

In [ ]:
%matplotlib inline
x_train.shape

In [ ]:
plt.imshow(x_train[0])

In [ ]:
# data_augmentation = keras.Sequential(
#     [
#         preprocessing.CenterCrop(img_height, img_width)
#     ]
# )

In [ ]:
# input
inp = tf.keras.Input(shape=(154, 154, 3))

# x = data_augmentation(inp)
# x = preprocessing.Rescaling(1.0 / 255)(x)

# convolutional layers
conv0 = tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation=tf.nn.relu,
)(inp)

conv1 = tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    activation=tf.nn.relu,
)(conv0)

# fully connected layer
flatten = tf.keras.layers.Flatten()(conv1)
dense = tf.keras.layers.Dense(units=2)(flatten)

model = tf.keras.Model(inputs=inp, outputs=dense)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

### Simple CNN

In [ ]:
history = model.fit(x_train,y_train,epochs = 10)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

# RNN

In [ ]:
x_train.shape

In [ ]:
n_steps = 10
train_sequences = np.tile(x_train[:, None], (1, n_steps,1, 1, 1))
test_sequences = np.tile(x_test[:, None], (1, n_steps,1, 1, 1))


In [ ]:
from tensorflow.keras.layers import Dense, Dropout, LSTM
model = tf.keras.Sequential()
model.add(tf.keras.layers.Reshape((-1, 256*256), input_shape=(None, 256, 256,3)),)
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,
             metrics=['accuracy'])

In [ ]:
model.fit(train_sequences,y_train, epochs = 10, batch_size = 4)

## ConvLSTM

In [ ]:
from tensorflow.keras import layers
inp = layers.Input(shape=(n_steps, *x_train.shape[1:]))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(5, 5),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(1, 1),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
flatten = tf.keras.layers.Flatten()(x)
dense = tf.keras.layers.Dense(units=2)(flatten)

model = tf.keras.Model(inputs=inp, outputs=dense)

# Next, we will build the complete model and compile it.
# model = tf.keras.models.Model(inp, x)
model.compile(
    loss=tf.keras.losses.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(),
)


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 10
batch_size = 2

# Fit the model to the training data.
model.fit(
    train_sequences,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
#     validation_data=(x_val, y_val),
#     callbacks=[early_stopping, reduce_lr],
)


### Spiking model

In [ ]:
train_sequences.shape

In [ ]:
spiking_model = tf.keras.Sequential(
    [
        # add temporal dimension to the input shape; we can set it to None,
        # to allow the model to flexibly run for different lengths of time
        tf.keras.layers.Reshape((-1, 154*154), input_shape=(None, 154, 154,3)),
        # we can use Keras' TimeDistributed wrapper to allow the Dense layer
        # to operate on temporal data
        
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(128)),
        # replace the "relu" activation in the non-spiking model with a
        # spiking equivalent
        keras_spiking.SpikingActivation("relu", spiking_aware_training=False),
        # use average pooling layer to average spiking output over time
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(2,activation='softmax'),
    ]
)




# input_l = tf.keras.layers.Input(shape=(None, img_height, img_width, 3))
# x = tf.keras.layers.TimeDistributed( tf.keras.layers.Conv2D( 32, kernel_size=(5, 5) ) ) (input_l)
# x = tf.keras.layers.TimeDistributed( tf.keras.layers.Conv2D(256, kernel_size=(5, 5)) ) (x)
# x = tf.keras.layers.Flatten()(x)
# out = tf.keras.layers.Dense(1)(x)
# spiking_model = models.Model(inputs=input_l, outputs=out)


def train(input_model, train_x, test_x):
    input_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )

    input_model.fit(train_x, y_train, epochs=10)

    _, test_acc = input_model.evaluate(test_x, y_test, verbose=2)

    print("\nTest accuracy:", test_acc)



# train the model, identically to the non-spiking version,
# except using the time sequences as inputs
train(spiking_model, train_sequences, test_sequences)


In [ ]:
spiking_model.summary()

In [ ]:
test_sequences.shape

In [ ]:
def check_output(seq_model, modify_dt=None):
    """
    This code is only used for plotting purposes, and isn't necessary to
    understand the rest of this example; feel free to skip it
    if you just want to see the results.
    """

    # rebuild the model with the functional API, so that we can
    # access the output of intermediate layers
    inp = x = tf.keras.Input(batch_shape=seq_model.layers[0].input_shape)

    has_global_average_pooling = False
    for layer in seq_model.layers:
        if isinstance(layer, tf.keras.layers.GlobalAveragePooling1D):
            # remove the pooling so that we can see the model's
            # output over time
            has_global_average_pooling = True
            continue

        if isinstance(layer, (keras_spiking.SpikingActivation, keras_spiking.Lowpass)):
            cfg = layer.get_config()
            # update dt, if specified
            if modify_dt is not None:
                cfg["dt"] = modify_dt
            # always return the full time series so we can visualize it
            cfg["return_sequences"] = True

            layer = type(layer)(**cfg)

        if isinstance(layer, keras_spiking.SpikingActivation):
            # save this layer so we can access it later
            spike_layer = layer

        x = layer(x)

    func_model = tf.keras.Model(inp, [x, spike_layer.output])

    # copy weights to new model
    func_model.set_weights(seq_model.get_weights())

    # run model
    output, spikes = func_model.predict(test_sequences)

    if has_global_average_pooling:
        # check test accuracy using average output over all timesteps
        predictions = np.argmax(output.mean(axis=1), axis=-1)
    else:
        # check test accuracy using output from only the last timestep
        predictions = np.argmax(output[:, -1], axis=-1)
    accuracy = np.equal(predictions, y_test).mean()
    print("Test accuracy: %.2f%%" % (100 * accuracy))

    time = test_sequences.shape[1] * spike_layer.dt
    n_spikes = spikes * spike_layer.dt
    rates = np.sum(n_spikes, axis=1) / time

    print(
        "Spike rate per neuron (Hz): min=%.2f mean=%.2f max=%.2f"
        % (np.min(rates), np.mean(rates), np.max(rates))
    )

    # plot output
    for ii in range(5):
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.title(class_names[y_test[ii]])
        plt.imshow(x_test[ii], cmap="gray")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.title("Spikes per neuron per timestep")
        bin_edges = np.arange(int(np.max(n_spikes[ii])) + 2) - 0.5
        plt.hist(np.ravel(n_spikes[ii]), bins=bin_edges)
        x_ticks = plt.xticks()[0]
        plt.xticks(
            x_ticks[(np.abs(x_ticks - np.round(x_ticks)) < 1e-8) & (x_ticks > -1e-8)]
        )
        plt.xlabel("# of spikes")
        plt.ylabel("Frequency")

        plt.subplot(1, 3, 3)
        plt.title("Output predictions")
        plt.plot(
            np.arange(test_sequences.shape[1]) * spike_layer.dt,
            tf.nn.softmax(output[ii][:10]),
        )
        plt.legend(class_names, loc="upper left")
        plt.xlabel("Time (s)")
        plt.ylabel("Probability")
        plt.ylim([-0.05, 1.05])

        plt.tight_layout()
check_output(spiking_model)

In [ ]:
check_output(spiking_model, modify_dt=0.01)

In [ ]:
check_output(spiking_model, modify_dt=0.1)

In [ ]:
check_output(spiking_model, modify_dt=1)

In [ ]:
spikeaware_model = tf.keras.Sequential(
    [
        tf.keras.layers.Reshape((-1, 154 * 154), input_shape=(None, 154,154,3)),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(128)),
        # set spiking_aware training and a moderate dt
        keras_spiking.SpikingActivation("relu", dt=0.01, spiking_aware_training=True),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(2),
    ]
)

train(spikeaware_model, train_sequences, test_sequences)



In [ ]:
check_output(spikeaware_model)

In [ ]:
keras_spiking.default.dt = 0.01

filtered_model = tf.keras.Sequential(
    [
        tf.keras.layers.Reshape((-1, 256 * 256), input_shape=(None, 256,256,3)),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(128)),
        keras_spiking.SpikingActivation("relu", spiking_aware_training=True),
        # add a lowpass filter on output of spiking layer
        keras_spiking.Lowpass(0.1, return_sequences=False),
        tf.keras.layers.Dense(10),
    ]
)

train(filtered_model, train_sequences, test_sequences)



In [ ]:
check_output(filtered_model)